In [1]:
def calculate_glcm_features(image, distances, angles):
    glcm_features = []

    for channel in range(image.shape[2]):
        channel_features = []

        for d in distances:
            for a in angles:
                glcm = greycomatrix(image[:, :, channel], [d], [a], 256, symmetric=True, normed=True)
                asm = greycoprops(glcm, 'ASM')[0][0]
                contrast = greycoprops(glcm, 'contrast')[0][0]
                correlation = greycoprops(glcm, 'correlation')[0][0]

                channel_features.extend([asm, contrast, correlation])

        glcm_features.append(channel_features)

    return np.array(glcm_features)

In [2]:
def extract_patches(mask, rgb, depth, area_threshold=100):
    # Convert mask to binary
    _, binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    # Find connected components
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_mask)

    # Initialize lists to store patches and GLCM features
    class1_patches = []
    class2_patches = []
    class1_glcm_features = []
    class2_glcm_features = []

    for label in range(1, num_labels):
        # Extract patch based on label
        patch_mask = (labels == label).astype(np.uint8)
        patch_rgb = cv2.bitwise_and(rgb, rgb, mask=patch_mask)
        patch_depth = cv2.bitwise_and(depth, depth, mask=patch_mask)

        # Calculate area of the patch
        patch_area = stats[label, cv2.CC_STAT_AREA]

        # Classify patches based on area
        if patch_area < area_threshold:
            class1_patches.append(patch_rgb)
            class1_glcm_features.append(calculate_glcm_features(patch_depth, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4]))
        else:
            class2_patches.append(patch_rgb)
            class2_glcm_features.append(calculate_glcm_features(patch_depth, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4]))

    return class1_patches, class2_patches, class1_glcm_features, class2_glcm_features